In [1]:
import os
import time
import random
import argparse
import numpy as np
from tqdm import tqdm
from mixup import mixup_data, mixup_criterion
#from utilities import AverageTracker, get_optimizer, showLR, CosineScheduler
from utilities import *
from pytorch_nn import Lipreading1
from datasetloadingpy import dataloaders

import torch
import torch.nn as nn
import torch.nn.functional as F

<font size='6'>
Contains First training loop for automation and testing training for 60 Epochs on larger portion of dataset
(For reasons behind Hyperparameter Choice in various_model_testing.ipynb)

In [2]:
def evaluate(model, dset_loader, criterion):

    model.eval()

    running_loss = 0.
    running_corrects = 0.

    with torch.no_grad():
        for batch_idx, (input, lengths, labels) in enumerate(tqdm(dset_loader)):
            logits = model(input.unsqueeze(1).cuda(), lengths=lengths)
            _, preds = torch.max(F.softmax(logits, dim=1).data, dim=1)
            running_corrects += preds.eq(labels.cuda().view_as(preds)).sum().item()

            loss = criterion(logits, labels.cuda())
            running_loss += loss.item() * input.size(0)

    print('{} in total\tCR: {}'.format( len(dset_loader.dataset), running_corrects/len(dset_loader.dataset)))
    return running_corrects/len(dset_loader.dataset), running_loss/len(dset_loader.dataset)

In [3]:
def train_loop(model, dataloader, criterion, epoch, optimizer, mixup=False):

    data_time = AverageTracker()
    batch_time = AverageTracker()

    mixup_alpha = 0.4

    print("Current Epoch: " + str(epoch))

    model.train()
    running_loss = 0.
    running_corrects = 0.
    running_all = 0.

    end = time.time()
    for batch_idx, (input, lengths, labels) in enumerate(dataloader):
        if mixup:
            input, labels_a, labels_b, lam = mixup_data(input, labels, mixup_alpha)
            labels_a, labels_b = labels_a.cuda(), labels_b.cuda()
        else:
            labels = labels.cuda()

        optimizer.zero_grad()

        logits = model(input.unsqueeze(1).cuda(), lengths=lengths)

        if mixup:
            loss_func = mixup_criterion(labels_a, labels_b, lam)
            loss = loss_func(criterion, logits)
        else:
            loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        # -- compute running performance
        _, predicted = torch.max(F.softmax(logits, dim=1).data, dim=1)
        running_loss += loss.item()*input.size(0)

        if mixup:
            running_corrects += lam * predicted.eq(labels_a.view_as(predicted)).sum().item() + (1 - lam) * predicted.eq(labels_b.view_as(predicted)).sum().item()
        else:
            running_corrects += torch.sum(predicted == labels.data)
        running_all += input.size(0)
    print("Running Loss: {}, Running Corrects: {}, Running All: {}".format(running_loss,running_corrects,running_all))

    return model

In [4]:
words_list_file = '/home/taylorpap/Bootcamp/wordlist.txt'
words_list = get_wordslist_from_txt_file(words_list_file)

In [5]:
temp_words_list = []
for index in range(1, len(words_list), 5):
    temp_words_list.append(words_list[index])
print(len(temp_words_list))

100


In [6]:
data_path = '/media/taylorpap/2TBM2/Dataset_2/CroppedLRW'
#temp_words_list = ['ABSOLUTELY', 'BUDGET', 'EVERYONE', 'HOUSE', 'MILITARY', 'PUBLIC', 'RESULT', 'SIGNIFICANT',
#                   'WEATHER']

datasets = dataloaders(data_dir=data_path, label_fp=temp_words_list, batch_size=32, workers=8)


Partition train loaded
Partition test loaded
Partition val loaded


In [7]:
train_features, train_lengths, train_labels = next(iter(datasets['train']))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 29, 88, 88])
Labels batch shape: torch.Size([32])


In [8]:
test_model = Lipreading1(len(temp_words_list))

In [9]:
if torch.cuda.is_available():
    test_model.cuda()

In [10]:
test_model

Lipreading1(
  (frontend3D): Sequential(
    (0): Conv3d(1, 32, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=32)
  )
  (frontend3D_2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (max_pool1): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
  (trunk): ResNet(
    (0): Sequential(
      (0): ResBlock(
        (convs): Sequential(
          (0): ConvLayer(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (1): ConvLayer(
            (0):

In [11]:
epochs = 60
lr = 3e-3
optimizer = get_optimizer('sgd', optim_policies=test_model.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()
scheduler = CosineScheduler(lr, epochs)
epoch=0

In [12]:
while epoch < epochs:
    start_epoch = time.time()
    model = train_loop(test_model, datasets['train'], criterion, epoch, optimizer)
    acc_avg_val, loss_avg_val = evaluate(model, datasets['val'], criterion)
    print('{} Epoch:\t{:2}\tLoss val: {:.4f}\tAcc val:{:.4f}, LR: {}'.format('val', epoch, loss_avg_val, acc_avg_val, showLR(optimizer)))
    save_dict = {
            'epoch_idx': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
        }
    scheduler.adjust_lr(optimizer, epoch)
    epoch +=1
    epoch_len = time.time() - start_epoch
    print("Epoch len: {} Estimated Remaining: {} Min".format(str(epoch_len), str(((epochs-epoch)*epoch_len)/60)))

acc_avg_test, loss_avg_test = evaluate(model, datasets['test'], criterion)
print('Test time performance of best epoch: {} (loss: {})'.format(acc_avg_test, loss_avg_test))

Current Epoch: 0
Running Loss: 447556.44184446335, Running Corrects: 1667.0, Running All: 97055.0


100%|██████████| 157/157 [00:08<00:00, 18.31it/s]

4996 in total	CR: 0.024619695756605286
val Epoch:	 0	Loss val: 4.5184	Acc val:0.0246, LR: 0.003
Epoch len: 607.9054090976715 Estimated Remaining: 597.773652279377 Min
Current Epoch: 1


Running Loss: 418696.5159883499, Running Corrects: 4581.0, Running All: 97055.0


100%|██████████| 157/157 [00:07<00:00, 20.59it/s]

...

Running Loss: 5401.3994647189975, Running Corrects: 95896.0, Running All: 97055.0


100%|██████████| 157/157 [00:06<00:00, 22.59it/s]


4996 in total	CR: 0.8799039231385108
val Epoch:	59	Loss val: 0.4672	Acc val:0.8799, LR: 8.2171569475899e-06
Epoch len: 575.379650592804 Estimated Remaining: 0.0 Min


100%|██████████| 157/157 [00:09<00:00, 16.94it/s]

4997 in total	CR: 0.8749249549729838
Test time performance of best epoch: 0.8749249549729838 (loss: 0.4989212736753348)


In [13]:
#ACC of 95.7777 on 9 Words ['ABSOLUTELY', 'BUDGET', 'EVERYONE', 'HOUSE', 'MILITARY', 'PUBLIC', 'RESULT', 'SIGNIFICANT', 'WEATHER']
#torch.save(model.state_dict(), '/home/taylorpap/Bootcamp/lipreadlstmv2.pth')

In [14]:
#ACC of 87 on 100 Words
torch.save(model.state_dict(), '/home/taylorpap/Bootcamp/lipreadlstmv4.pth')

<font size='6'>
With 100 Word Dataset, First Model managed to reach 87% after nearly 10 hours of training. Will need to use reduced dataset size (Lower number of words) and fewer epochs to test multiple different models for performance.